In [28]:
import requests
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

def fetch_moex_tickers():
    url = 'https://iss.moex.com/iss/engines/stock/markets/shares/securities.json'
    response = requests.get(url)
    data = response.json()
    securities = data['securities']['data']
    columns = data['securities']['columns']
    df = pd.DataFrame(securities, columns=columns)
    df_filtered = df[['SECID', 'SHORTNAME', 'SECNAME', 'LATNAME']].dropna()
    df_filtered = df_filtered[df_filtered['SECID'].apply(lambda x: len(str(x)) <= 6)]
    company_to_ticker = {}
    for _, row in df_filtered.iterrows():
        names = [row['SECNAME'], row['SHORTNAME'], row['LATNAME']]
        for name in names:
            company_to_ticker[name.lower()] = row['SECID']
    return company_to_ticker

model_name = "sberbank-ai/ruBert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :].numpy()
    return embedding

company_to_ticker = fetch_moex_tickers()
company_names = list(company_to_ticker.keys())
embeddings = [get_embedding(name)[0] for name in company_names]

target_name = "Газпром"
target_embedding = get_embedding(target_name)[0]
similarities = cosine_similarity([target_embedding], embeddings)[0]
best_idx = similarities.argmax()
best_match_name = company_names[best_idx]
best_match_ticker = company_to_ticker[best_match_name]

print(f"Best match: {best_match_name}")
print(f"Ticker: {best_match_ticker}")
print(f"Cosine similarity: {similarities[best_idx]:.4f}")


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Best match: роснефть
Ticker: ROSN
Cosine similarity: 0.9045


In [236]:
df_filtered = pd.read_excel(r"C:\Users\Ольга\ВКР Карпенко\data markup\filtered_with_tickers_2.xlsx")
df_filtered

,Текст,Ticker,Correct
0,"🍷**EBITDA LTM ""Новабев групп"" на 30 июня 2024 ...",BELU,1
1,"🔺**""Соллерс"" поднял пороговое значение учитыва...",SVAV,1
2,"🗓**""НОВАТЭК"" 6 февраля проведет сбор заявок на...",NVTK,1
3,"**Минфин не слышал о планах ""Транснефти"" снизи...",TRNFP,1
4,**💰SoftBank Group ведет переговоры об инвестиц...,NaN,0
...,...,...,...
374,**Роснано допустило техдефолт по купону облига...,NaN,1
375,"🗓**Совет директоров ""Новабев Групп"" 2 апреля р...",BELU,1
376,"**""Циан"" одновременно со стартом торгов на Мос...",CNRU,1
377,"**""Циан"" в IV квартале нарастил выручку на 5%,...",CNRU,1


## testing re + rapidfuzz

In [246]:
import re

df_filtered['company'] = df_filtered['Текст'].apply(lambda text: re.findall(r'«(.*?)»|"(.*?)"|“(.*?)”|\((.*?)\)', text))

df_filtered['company'] = df_filtered['company'].apply(lambda matches: [match for group in matches for match in group if match])

df_filtered =  df_filtered[['Текст', 'company', 'Ticker']]
df_filtered

,Текст,company,Ticker
0,"🍷**EBITDA LTM ""Новабев групп"" на 30 июня 2024 ...","[Новабев групп, Новабев Групп]",BELU
1,"🔺**""Соллерс"" поднял пороговое значение учитыва...","[Соллерс, чистый долг/EBITDA, Соллерс, чистый ...",SVAV
2,"🗓**""НОВАТЭК"" 6 февраля проведет сбор заявок на...","[НОВАТЭК, НОВАТЭК, Интерфаксу, Ньютон инвестиции]",NVTK
3,"**Минфин не слышал о планах ""Транснефти"" снизи...","[Транснефти, Транснефти, Я не слышал о таком, ...",TRNFP
4,**💰SoftBank Group ведет переговоры об инвестиц...,"[более $15 млрд, Переговоры идут, и сумма, кот...",NaN
...,...,...,...
374,**Роснано допустило техдефолт по купону облига...,"[Роснано, разработка технического механизма вы...",NaN
375,"🗓**Совет директоров ""Новабев Групп"" 2 апреля р...","[Новабев Групп, Рассмотрение возможности прове...",BELU
376,"**""Циан"" одновременно со стартом торгов на Мос...","[Циан, Циан, Циан, Циан, Циан, Циан]",CNRU
377,"**""Циан"" в IV квартале нарастил выручку на 5%,...","[Циан, Циан, Интерфакса, Циан]",CNRU


In [247]:
import requests
import pandas as pd
from rapidfuzz import fuzz

def fetch_moex_tickers():
    url = 'https://iss.moex.com/iss/engines/stock/markets/shares/securities.json'
    response = requests.get(url)
    data = response.json()
    securities = data['securities']['data']
    columns = data['securities']['columns']
    df = pd.DataFrame(securities, columns=columns)
    df_filtered = df[['SECID', 'SHORTNAME', 'SECNAME', 'LATNAME']].dropna()
    df_filtered = df_filtered[df_filtered['SECID'].apply(lambda x: len(str(x)) <= 6)]
    company_to_ticker = {}
    for _, row in df_filtered.iterrows():
        names = [row['SECNAME'], row['SHORTNAME'], row['LATNAME']]
        for name in names:
            company_to_ticker[name.lower()] = row['SECID']
    return company_to_ticker

company_to_ticker = fetch_moex_tickers()
company_names = list(company_to_ticker.keys())

def find_ticker_by_top3(companies):
    for company in companies[:3]:  
        best_match_name = None
        best_score = 0
        for name in company_names:
            score = fuzz.partial_ratio(company.lower(), name.lower())
            if score > best_score:
                best_score = score
                best_match_name = name
        if best_score >= 85:  
            return company_to_ticker[best_match_name]
    return None

df_filtered['Predicted'] = df_filtered['company'].apply(find_ticker_by_top3)

df_filtered

C:\Users\Ольга\AppData\Local\Temp\ipykernel_12676\4091543507.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Predicted'] = df_filtered['company'].apply(find_ticker_by_top3)


,Текст,company,Ticker,Predicted
0,"🍷**EBITDA LTM ""Новабев групп"" на 30 июня 2024 ...","[Новабев групп, Новабев Групп]",BELU,BELU
1,"🔺**""Соллерс"" поднял пороговое значение учитыва...","[Соллерс, чистый долг/EBITDA, Соллерс, чистый ...",SVAV,SVAV
2,"🗓**""НОВАТЭК"" 6 февраля проведет сбор заявок на...","[НОВАТЭК, НОВАТЭК, Интерфаксу, Ньютон инвестиции]",NVTK,NVTK
3,"**Минфин не слышал о планах ""Транснефти"" снизи...","[Транснефти, Транснефти, Я не слышал о таком, ...",TRNFP,TRNFP
4,**💰SoftBank Group ведет переговоры об инвестиц...,"[более $15 млрд, Переговоры идут, и сумма, кот...",NaN,None
...,...,...,...,...
374,**Роснано допустило техдефолт по купону облига...,"[Роснано, разработка технического механизма вы...",NaN,None
375,"🗓**Совет директоров ""Новабев Групп"" 2 апреля р...","[Новабев Групп, Рассмотрение возможности прове...",BELU,BELU
376,"**""Циан"" одновременно со стартом торгов на Мос...","[Циан, Циан, Циан, Циан, Циан, Циан]",CNRU,CNRU
377,"**""Циан"" в IV квартале нарастил выручку на 5%,...","[Циан, Циан, Интерфакса, Циан]",CNRU,CNRU


In [59]:
df_filtered.to_excel('df_filtered.xlsx', index=False)

In [248]:
if 'Ticker' in df_filtered.columns and 'Predicted' in df_filtered.columns:
    df_filtered['is_correct'] = df_filtered.apply(
        lambda row: (row['Ticker'] == row['Predicted']) or 
                    (pd.isna(row['Ticker']) and pd.isna(row['Predicted'])), axis=1)
    total = len(df_filtered)
    correct = df_filtered['is_correct'].sum()
    accuracy = correct / total if total > 0 else 0
    print(f"Accuracy of ticker matching: {accuracy:.4f}")
    print(f"Out of {total} records, {correct} were predicted correctly.")
else:
    print("Columns 'Ticker' and 'Predicted' were not found in the DataFrame.")

Accuracy of ticker matching: 0.7414
Out of 379 records, 281 were predicted correctly.


## Step 2 NER + TF IDF

In [237]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsNERTagger,
    Doc
)

segmenter = Segmenter()
emb = NewsEmbedding()
ner_tagger = NewsNERTagger(emb)

def extract_companies_natasha(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)
    companies = [span.text for span in doc.spans if span.type == 'ORG']
    
    return companies

df_filtered['company_natasha'] = df_filtered['Текст'].apply(extract_companies_natasha)


df_companies_natasha = df_filtered[['Текст', 'company_natasha', 'Ticker']].copy()

df_companies_natasha['company_natasha'] = df_companies_natasha['company_natasha'].apply(
    lambda companies: ['Т-технологии' if ('Росбанк' in c or 'T-банк' in c) else c for c in companies]
)

df_companies_natasha

,Текст,company_natasha,Ticker
0,"🍷**EBITDA LTM ""Новабев групп"" на 30 июня 2024 ...","[Новабев групп, Новабев Групп, LTM, LTM]",BELU
1,"🔺**""Соллерс"" поднял пороговое значение учитыва...","[Соллерс, ПАО ""Соллерс"", Соллерса, Соллерс]",SVAV
2,"🗓**""НОВАТЭК"" 6 февраля проведет сбор заявок на...","[НОВАТЭК, ПАО ""НОВАТЭК"", Интерфаксу, Банка Рос...",NVTK
3,"**Минфин не слышал о планах ""Транснефти"" снизи...","[Минфин, Транснефти, Министерство финансов, Тр...",TRNFP
4,**💰SoftBank Group ведет переговоры об инвестиц...,"[SoftBank Group, Financial Times, OpenAI, Soft...",NaN
...,...,...,...
374,**Роснано допустило техдефолт по купону облига...,"[Роснано, АО ""Роснано"", Роснано, Интерфакс, Ро...",NaN
375,"🗓**Совет директоров ""Новабев Групп"" 2 апреля р...","[Новабев Групп, Winelab**, ВинЛаб]",BELU
376,"**""Циан"" одновременно со стартом торгов на Мос...","[Циан, МКПАО, МКПАО ""Циан, Циан, МКПАО]",CNRU
377,"**""Циан"" в IV квартале нарастил выручку на 5%,...","[Интерфакса, Циан]",CNRU


In [241]:
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def fetch_moex_tickers():
    url = 'https://iss.moex.com/iss/engines/stock/markets/shares/securities.json'
    response = requests.get(url)
    data = response.json()
    securities = data['securities']['data']
    columns = data['securities']['columns']
    df = pd.DataFrame(securities, columns=columns)
    df_filtered = df[['SECID', 'SHORTNAME', 'SECNAME', 'LATNAME']].dropna()
    df_filtered = df_filtered[df_filtered['SECID'].apply(lambda x: len(str(x)) <= 6)]
    company_to_ticker = {}
    for _, row in df_filtered.iterrows():
        names = [row['SECNAME'], row['SHORTNAME'], row['LATNAME']]
        for name in names:
            company_to_ticker[name.lower()] = row['SECID']
    
    company_to_ticker.update({
        'иват': 'IVAT',
        'iva technologies': 'IVAT',
        'норникель': 'GMKN',
        'норильский никель': 'GMKN',
        'fix price': 'FIXP',
        'фикс прайс': 'FIXP',
        'cмарттехгрупп': 'CARM',
        "о'кей": 'OKEY'
    })
    
    remove_words = ['холдинг', 'holding', 'интернешионал', 'international']
    cleaned_company_to_ticker = {}
    for key, value in company_to_ticker.items():
        cleaned_key = key
        for word in remove_words:
            cleaned_key = cleaned_key.replace(word, '').strip()
        cleaned_company_to_ticker[cleaned_key] = value
    
    return cleaned_company_to_ticker


company_to_ticker = fetch_moex_tickers()
company_names = list(company_to_ticker.keys())
vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(3,5))
tfidf_matrix = vectorizer.fit_transform(company_names)

def find_ticker_by_top3(companies):
    for company in companies[:3]:
        org_norm = company.lower()
        
        target_vector = vectorizer.transform([org_norm])
        similarities = cosine_similarity(target_vector, tfidf_matrix)[0]
        best_idx = similarities.argmax()
        best_similarity = similarities[best_idx]
        if best_similarity >= 0.5:
            best_match_name = company_names[best_idx]
            return company_to_ticker[best_match_name]
    return None


df_companies_natasha['Predicted'] = df_companies_natasha['company_natasha'].apply(find_ticker_by_top3)

df_companies_natasha

,Текст,company_natasha,Ticker,Predicted,is_correct
0,"🍷**EBITDA LTM ""Новабев групп"" на 30 июня 2024 ...","[Новабев групп, Новабев Групп, LTM, LTM]",BELU,BELU,True
1,"🔺**""Соллерс"" поднял пороговое значение учитыва...","[Соллерс, ПАО ""Соллерс"", Соллерса, Соллерс]",SVAV,SVAV,True
2,"🗓**""НОВАТЭК"" 6 февраля проведет сбор заявок на...","[НОВАТЭК, ПАО ""НОВАТЭК"", Интерфаксу, Банка Рос...",NVTK,NVTK,True
3,"**Минфин не слышал о планах ""Транснефти"" снизи...","[Минфин, Транснефти, Министерство финансов, Тр...",TRNFP,TRNFP,True
4,**💰SoftBank Group ведет переговоры об инвестиц...,"[SoftBank Group, Financial Times, OpenAI, Soft...",NaN,None,True
...,...,...,...,...,...
374,**Роснано допустило техдефолт по купону облига...,"[Роснано, АО ""Роснано"", Роснано, Интерфакс, Ро...",NaN,None,True
375,"🗓**Совет директоров ""Новабев Групп"" 2 апреля р...","[Новабев Групп, Winelab**, ВинЛаб]",BELU,BELU,True
376,"**""Циан"" одновременно со стартом торгов на Мос...","[Циан, МКПАО, МКПАО ""Циан, Циан, МКПАО]",CNRU,CNRU,True
377,"**""Циан"" в IV квартале нарастил выручку на 5%,...","[Интерфакса, Циан]",CNRU,CNRU,True


In [242]:
if 'Ticker' in df_companies_natasha.columns and 'Predicted' in df_companies_natasha.columns:
    df_companies_natasha['is_correct'] = df_companies_natasha.apply(
        lambda row: (row['Ticker'] == row['Predicted']) or 
                    (pd.isna(row['Ticker']) and pd.isna(row['Predicted'])), axis=1)
    total = len(df_companies_natasha)
    correct = df_companies_natasha['is_correct'].sum()
    accuracy = correct / total if total > 0 else 0
    print(f"Accuracy of ticker matching: {accuracy:.4f}")
    print(f"Out of {total} records, {correct} were predicted correctly.")
else:
    print("Columns 'Ticker' and 'Predicted' were not found in the DataFrame.")


Accuracy of ticker matching: 0.9472
Out of 379 records, 359 were predicted correctly.


In [243]:
if 'Ticker' in df_companies_natasha.columns and 'Predicted' in df_companies_natasha.columns:
    df_error = df_companies_natasha[~(
        (df_companies_natasha['Ticker'] == df_companies_natasha['Predicted']) |
        ((df_companies_natasha['Ticker'].fillna(0) == 0) & (df_companies_natasha['Predicted'].fillna(0) == 0))
    )]
df_error

,Текст,company_natasha,Ticker,Predicted,is_correct
22,**Skillbox Holding редомицилирован в РФ**\n\nS...,"[Skillbox Holding Limited, ЕГРЮЛ, МКАО ""Скилбо...",T,None,False
33,**🏦BNY Mellon в 2024 году получил рекордную чи...,[Bank of New York Mellon],NaN,MBNK,False
34,"**ЦБ РФ зарегистрировал выпуск акций МКАО ""Ски...","[ЦБ, МКАО ""Скилбокс Холдинг, Банк России, МКАО...",WUSH,SBER,False
45,**Сбер восстановил достаточность капитала до н...,[],SBER,None,False
47,**⚖️Суд отказал в частичной отмене обеспечител...,"[Арбитражный суд, Борец, ООО "" Борец Капитал, ...",0,AKGD,False
62,"**ХК ""Металлоинвест"" зафиксировала объем разме...","[ХК ""Металлоинвест"", ХК ""Металлоинвест"", Интер...",NaN,IRAO,False
67,**Промсвязьбанк зафиксировал объем размещения ...,"[Промсвязьбанк, ПАО ""Промсвязьбанк"", Интерфакс...",PSGM,IRAO,False
109,**JetLend в рамках IPO привлек чуть меньше 500...,"[ПАО ""ДжетЛенд Холдинг, ДжетЛенд Холдинга, СПБ...",NaN,SPBE,False
118,"**Акционеры ГК ""О'Кей"" утвердили решение о ред...","[ГК ""О'Кей, О'Кей, О'кей Груп, МКПАО]",NaN,OKEY,False
122,"**ФПК ""Гарант-инвест"" допустила техдефолт по в...","[ФПК ""Гарант-инвест"", ЦБ, Гарант-инвест]",NaN,RUSI,False
